In [1]:
from django.db import models
import json
class Block(models.Model):
    index = models.IntegerField()
    timestamp = models.DateTimeField(auto_now_add=True)
    previous_hash = models.CharField(max_length=64)
    proof = models.CharField(max_length=256)
    data = models.TextField()  # Storing JSON as text
    hash = models.CharField(max_length=64)

    def __str__(self):
        return f'Block {self.index} with hash {self.hash}'

    def get_data(self):
        return json.loads(self.data)

    def set_data(self, data_dict):
        self.data = json.dumps(data_dict)

ImproperlyConfigured: Requested setting INSTALLED_APPS, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.

In [2]:
import hashlib
from hashlib import sha256
def compute_hash(block):
    """
    Compute the SHA-256 hash of the given block.
    """
    block_string = json.dumps({
        'index': block.index,
        'timestamp': block.timestamp,
        'previous_hash': block.previous_hash,
        'proof': block.proof,
        'data': block.data
    }, sort_keys=True).encode()
    
    return hashlib.sha256(block_string).hexdigest()

def proof_of_work(previous_proof):
    """
    Simple proof of work algorithm:
    - Find a number p such that hash(p * 2 + previous_proof) has 4 leading zeros.
    """
    proof = 0
    while True:
        guess = f'{proof}{previous_proof}'.encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        if guess_hash[:4] == "0000":  # Adjust the number of leading zeros as needed
            return proof
        proof += 1

def create_blockchain_entry(unit, block_data):
    

    # Get the previous block
    previous_block = Block.objects.last()
    previous_hash = previous_block.hash if previous_block else '0'

    # Run proof of work
    previous_proof = previous_block.proof if previous_block else 0
    proof = proof_of_work(previous_proof)

    # Create a new block with the computed hash and proof
    block = Block(
        index=(previous_block.index + 1 if previous_block else 1),
        timestamp=block_data.get('timestamp'),
        previous_hash=previous_hash,
        proof=proof,
        data=json.dumps(block_data, sort_keys=True)
    )
    

    # Compute the hash for the new block
    block.hash = compute_hash(block)

    # Save the block to the database
    block.save()


def verify_unit_authenticity(unit_id):
    # Retrieve all blocks related to the given unit_id
    blocks = Block.objects.all()

    # Initialize an empty list to store the transaction history for the unit
    transaction_history = []

    # Validate the entire blockchain
    previous_hash = "0"
    for block in blocks:
        # Recompute the hash of the block
        block_obj = Block(
                        index=(block.index),
                        timestamp=str(block.timestamp.isoformat()),
                        previous_hash=previous_hash,
                        proof=block.proof,
                        data=block.data,
                        )
    
        computed_hash = compute_hash(block_obj)

        # Check if the computed hash matches the stored hash
        if block.hash != computed_hash:
            return False, "Blockchain integrity check failed.", None

        # Check if the previous_hash matches the hash of the previous block
        if block.previous_hash != previous_hash:
            return False, "Blockchain chain is broken.", None

        # Update previous_hash for the next iteration
        previous_hash = block.hash

        # Check for unit_id in the block's data
        block_data_dict = json.loads(block.data)
        if block_data_dict.get("unit_id") == unit_id:
            transaction_history.append({
                "action": block_data_dict.get("action"),
                "timestamp": block_data_dict.get("timestamp"),
                "block_index": block.index
            })

    # If transaction history is found for the unit_id
    if transaction_history:
        # Retrieve the unit and associated product details
        unit = Unit(.objects.get(id=unit_id))
        product = unit.product
        manufacturer = product.manufacturer

        # Prepare the result data
        unit_data = {
            "unit_id": unit.id,
            "product_name": product.name,
            "manufacturer_name": manufacturer.name,
            "transaction_history": transaction_history
        }
        return True, "Unit is authentic.", unit_data
    else:
        return False, "Unit not found in the blockchain.", None

SyntaxError: invalid syntax (933503448.py, line 102)

In [ ]:
previous_proof = 0
proof = proof_of_work(previous_proof)

block = Block(
            index=1,
            timestamp=str(block.timestamp.isoformat()),
            previous_hash=0,
            proof=proof,
            data=block.data,
            )
    

    # Compute the hash for the new block
block.hash = compute_hash(block)

print(block.hash)

In [1]:
import hashlib
import json
from datetime import datetime

# Define Block class
class Block:
    def __init__(self, index, timestamp, previous_hash, proof, data):
        self.index = index
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.proof = proof
        self.data = data
        self.hash = self.compute_hash()

    def compute_hash(self):
        block_string = json.dumps({
            'index': self.index,
            'timestamp': self.timestamp.isoformat() if isinstance(self.timestamp, datetime) else self.timestamp,
            'previous_hash': self.previous_hash,
            'proof': self.proof,
            'data': self.data
        }, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

# Define Proof of Work function
def proof_of_work(previous_proof):
    proof = 0
    while True:
        guess = f'{proof}{previous_proof}'.encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        if guess_hash[:4] == "0000":  # Example of proof of work with 4 leading zeros
            return proof
        proof += 1

# Sample Blockchain Initialization
def create_genesis_block():
    return Block(index=1, timestamp=datetime.now(), previous_hash='0', proof=100, data=json.dumps({"action": "genesis"}))

def create_blockchain_entry(previous_block, action, unit_id):
    block_data = {
        'unit_id': unit_id,
        'action': action,
        'timestamp': datetime.now().isoformat(),
    }

    proof = proof_of_work(previous_block.proof)
    new_block = Block(
        index=previous_block.index + 1,
        timestamp=datetime.now(),
        previous_hash=previous_block.hash,
        proof=proof,
        data=json.dumps(block_data)
    )
    
    return new_block

# Initialize the blockchain with a genesis block
genesis_block = create_genesis_block()
blockchain = [genesis_block]

# Create new blocks
block1 = create_blockchain_entry(genesis_block, "Unit added", "unit_1")
block2 = create_blockchain_entry(block1, "Unit shipped", "unit_1")

blockchain.extend([block1, block2])

# Print block details
for block in blockchain:
    print(f"Block {block.index}:")
    print(f"Hash: {block.hash}")
    print(f"Data: {block.data}")
    print()


Block 1:
Hash: ceb32e0500b67d802efe2473954087731c1682d0991569394751da738d6f3dea
Data: {"action": "genesis"}

Block 2:
Hash: 249a48d823b77e28cacc4313c4648c5d1b23310be2168fccfb587e35ff233b47
Data: {"unit_id": "unit_1", "action": "Unit added", "timestamp": "2024-08-25T19:43:03.816854"}

Block 3:
Hash: 0d165f3d9f619f37954d9edf7af2ac3f260e5406012ced08097c2d2cb192b5ad
Data: {"unit_id": "unit_1", "action": "Unit shipped", "timestamp": "2024-08-25T19:43:04.112996"}



In [5]:
def verify_blockchain(blockchain):
    previous_hash = '0'
    for block in blockchain:
        # Check hash
        if block.hash != block.compute_hash():
            return False, f"Block {block.index} hash is invalid"

        # Check previous hash
        if block.previous_hash != previous_hash:
            return False, "Blockchain chain is broken."

        previous_hash = block.hash
    
    return True, "Blockchain is valid."

def verify_unit_authenticity(blockchain, unit_id):
    is_valid, message = verify_blockchain(blockchain)
    if not is_valid:
        return False, message, None

    transaction_history = []
    for block in blockchain:
        block_data_dict = json.loads(block.data)
        if block_data_dict.get("unit_id") == unit_id:
            transaction_history.append({
                "action": block_data_dict.get("action"),
                "timestamp": block.timestamp,
                "block_index": block.index
            })

    if transaction_history:
        return True, "Unit is authentic.", transaction_history
    else:
        return False, "Unit not found in the blockchain.", None

# Test the verification function
unit_id = "unit_1"
is_authentic, message, transaction_history = verify_unit_authenticity(blockchain, unit_id)

print(message)
if is_authentic:
    for tx in transaction_history:
        print(f"Block Index: {tx['block_index']}, Action: {tx['action']}, Timestamp: {tx['timestamp']}")

Unit is authentic.
Block Index: 2, Action: Unit added, Timestamp: 2024-08-25 19:43:04.112996
Block Index: 3, Action: Unit shipped, Timestamp: 2024-08-25 19:43:04.246923
